<a href="https://colab.research.google.com/github/eldend/kh/blob/main/%EA%B3%BC%EC%A0%9C1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#--과제1--
타이타닉 데이터 세트를 이용하여

*   SGD 분류
*   결정트리
*   랜덤포레스트
*   히스토그램기반 그래디언트 부스팅 모델을 만들고 최적화

In [14]:
import pandas as pd
import pandas
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier # 결정트리
from sklearn.preprocessing import StandardScaler #정규화
from sklearn.linear_model import SGDClassifier #SGD 분류
from sklearn.ensemble import RandomForestClassifier # 랜덤포레스트
from sklearn.ensemble import HistGradientBoostingClassifier # 그래디언트 부스팅
#from sklearn.model_selection import train_test_split -> 데이터가 이미 나눠져 있기 때문에 나눌 필요가 없음
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test.csv')
#Age와 Cabin, Embarked에 Nan값이 존재-> Age만 사용할 것이기 때문에 Age만 Nan값을 채움
#Age의 경우 편균값으로 변경
train['Age'].fillna(train['Age'].mean(), inplace=True)
train.isnull().sum().sum()
test['Age'].fillna(test['Age'].mean(), inplace=True)
test.isnull().sum().sum()
#불필요한 필드 삭제(생존에 영향을 미치지 않음)
train = train.drop(['Ticket', 'Cabin', 'Name', 'Parch', 'SibSp', 'Embarked', 'Fare', 'PassengerId'], axis=1)
test = test.drop(['Ticket', 'Cabin', 'Name', 'Parch', 'SibSp', 'Embarked', 'Fare'], axis=1)
#데이터 결합
combine = [train, test]
print(combine)
#성별의 경우 데이터가 string형태이기때문에 여자면 1, 남잠녀 0으로 데이터형식을 변환
for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)
#나이의 경우 범위가 넓기 때문에 10대 20대와 같이 분류
for dataset in combine:
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age'] = 4
#데이터 나누기 -> (정보, 생존여부)와 같은 형태를 만듦
X_train = train.drop(["Survived"], axis=1)
Y_train = train["Survived"]
X_test  = test.copy().drop("PassengerId", axis=1)
#SGD모델 실행
sgd = SGDClassifier()
sgd.fit(X_train, Y_train) #-> (정보, 생존 여부)
Y_pred = sgd.predict(X_test)
acc_sgd = sgd.score(X_train, Y_train)
print('SGD Accuracy: ', acc_sgd)
#결정트리 모델 실행
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree = decision_tree.score(X_train, Y_train)
print('결정트리 Accuracy: ', acc_decision_tree)
#랜덤포레스트 모델 실행
random_forest = RandomForestClassifier(n_jobs= -1, n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = random_forest.score(X_train, Y_train)
print('랜덤포레스트 Accuracy: ', acc_random_forest)
#히스토리 기반 그레디언트 부스팅 모델 실행
hgb = HistGradientBoostingClassifier(random_state=202139820)
hgb.fit(X_train, Y_train)
Y_pred = hgb.predict(X_test)
acc_hgb = hgb.score(X_train, Y_train)
print('히스토기반 그레디언트 부스팅 Accuracy: ', acc_hgb)

[     Survived  Pclass     Sex        Age
0           0       3    male  22.000000
1           1       1  female  38.000000
2           1       3  female  26.000000
3           1       1  female  35.000000
4           0       3    male  35.000000
..        ...     ...     ...        ...
886         0       2    male  27.000000
887         1       1  female  19.000000
888         0       3  female  29.699118
889         1       1    male  26.000000
890         0       3    male  32.000000

[891 rows x 4 columns],      PassengerId  Pclass     Sex       Age
0            892       3    male  34.50000
1            893       3  female  47.00000
2            894       2    male  62.00000
3            895       3    male  27.00000
4            896       3  female  22.00000
..           ...     ...     ...       ...
413         1305       3    male  30.27259
414         1306       1  female  39.00000
415         1307       3    male  38.50000
416         1308       3    male  30.27259
417      